# 编写python爬虫，爬取新浪新闻内容

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json

## 1.具体抓取一个新闻的评论数

In [5]:
'''
功能：从具体网址中提取评论数
参数：newsurl——当前具体新闻url（首页-新闻版块-该新闻具体网址）
返回：评论数(数字)
'''
#commentURL = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=gn&newsid=comos-{}&group=&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=20&jsvar=loader_1503826269048_18480024'
#由于我们之后（只要是新闻评论都说固定的）
commentURL = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=gn&newsid=comos-{}&group=&compress=0&ie=utf-8&oe=utf-8'
def getCommentCounts(newsurl):
    m = re.search('doc-i(.+).shtml', newsurl) #从当前url查找新闻id
    newsid = m.group(1) #()内的内容
    comments = requests.get(commentURL.format(newsid)) #拼接评论网址，读取评论网页
    jd = json.loads(comments.text.strip('var data='))
    return jd['result']['count']['total']

In [6]:
#当前新闻url
newsurl = 'http://news.sina.com.cn/o/2017-09-06/doc-ifykpzey4646471.shtml'
#存放评论的url
commentURL = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=gn&newsid=comos-{}&group=&compress=0&ie=utf-8&oe=utf-8'
getCommentCounts(newsurl)

93

## 2.具体抓取一个新闻的所有内容

In [8]:
from datetime import datetime 

In [9]:
'''
功能：从具体网址中提取需要的所有内容
实现：
1-打开网址，用soup读取
2-具体提取各个部分的内容
参数：newsurl——当前具体新闻url（首页-新闻版块-该新闻具体网址）
返回：内容字典
'''
def getNewsDetail(newsurl):
    #1-打开网址，用soup读取
    res = requests.get(newsurl)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'html.parser')
    #2-具体提取各个部分的内容
    result={}
    result['title'] = soup.select('#artibodyTitle')[0].text      #标题
    #时间
    timesource=soup.select('.time-source')[0].contents[0].strip()
    result['dt'] = datetime.strptime(timesource,'%Y年%m月%d日%H:%M')
    result['source'] = soup.select('.time-source span a')[0].text  #来源
    #内文
    result['article'] = ' '.join([p.text.strip() for p in soup.select('#artibody p')[:-1]])
    #编辑
    result['editor'] = soup.select('.article-editor')[0].text.lstrip('责任编辑：').strip()
    #评论数
    result['comments'] = getCommentCounts(newsurl)
    return result

In [19]:
#getNewsDetail(newsurl)

## 3. 按列表进行抓取新闻
重点：从（首页-新闻版块）中的列表可以得到所有新闻的url地址

In [21]:
'''
功能：由传入url，获得所有新闻
步骤：
1-由JS的列表，获得列表内的所有新闻url地址
2-将所有地址都抓取具体内容，以列表的形式存放
参数：每页的Js列表对应的url
url = http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1
返回：列表（含多个字典）
'''   
def parseListLinks(url):
    newsdetails = []
    res = requests.get(url)
    jd = json.loads(res.text)
    for ent in jd['result']['data']:
        newsdetails.append(getNewsDetail(ent['url'])) 
    return newsdetails

In [32]:
#产生多页内容
url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}'
news_total=[]
for i in range(1,3):
    newsurl = url.format(i)
    news_total.extend(parseListLinks(newsurl))  
#news_total

## 4. 使用pandas整理数据

In [12]:
import pandas as pd

In [34]:
df = pd.DataFrame(news_total)
df

,article,comments,dt,editor,source,title
0,原标题：南京大学党委书记寄语新生：在平凡的每一天思考不平凡的事 除了日常的哲学授课以外，作为...,0,2017-09-06 13:48:00,张建利,澎湃新闻,南大党委书记寄语新生:平凡的每天思考不平凡事
1,原标题：台媒：大陆完成全球首个海水量子通信实验 上天入地下海可期 参考消息网9月6日报道 台...,0,2017-09-06 13:44:00,初晓慧,参考消息,中国完成首个海水量子通信实验 上天下海可期
2,原标题：蔡英文男女侍卫曝性丑闻 处理：女的走人 男的留下 海外网9月6日电 据台湾“中央社”...,1,2017-09-06 13:48:00,初晓慧,人民日报海外版-海外网,蔡英文男女侍卫曝性丑闻被处理:女走男留
3,原标题：黄兰香看望袁隆平院士 今年9月7日是“杂交水稻之父”袁隆平院士88岁（虚岁）生日。9...,0,2017-09-06 13:45:00,张建利,澎湃新闻,袁隆平将年满88岁 湖南统战部长看望并送上祝福
4,原标题：重大突破！我国首次钻获温度最高的干热岩 或将改写传统能源版图！ 记者从国土资源部中国...,0,2017-09-06 13:28:00,张建利,央视新闻,中国首次钻获温度最高干热岩 或改写能源版图
5,原标题：湖北蕲春通报：豪赌视频6人中3人系党员干部，警方纪委介入 针对微信朋友圈疯传湖北黄冈...,0,2017-09-06 13:22:00,桂强,澎湃新闻,湖北蕲春通报:豪赌视频中3人系干部 纪委已介入
6,原标题：突发，这类人可“无条件落户”！2.8亿人沸腾了 户口簿，平时似乎一直躺抽屉，如果一旦...,0,2017-09-06 13:12:00,张建利,央视财经,浙江出台户口迁移政策:优秀农民工可无条件落户
7,中新网9月6日电 据中国气象局网站消息，中央气象台9月6日10时发布台风蓝色预警。今年第17...,0,2017-09-06 13:03:00,张建利,中国新闻网,台风蓝色预警：“古超”将移入台湾海峡南部
8,原标题：江苏省人民检察院依法对吴志刚（正厅级）以涉嫌受贿罪立案侦查 江苏检察在线消息，日前，...,4,2017-09-06 13:10:00,桂强,澎湃新闻,江苏徐州矿务集团原董事长吴志刚被立案侦查
9,原标题：台名校女高中生跳楼身亡 留下遗书：不用找了 就是我 据台湾“中时电子报”9月6日报道...,0,2017-09-06 12:46:00,初晓慧,环球网,台名校女高中生跳楼身亡留遗书:不用找了就是我


In [36]:
#保存数据
df.to_excel('sinaCrawlerDemo.xlsx')